## 2. Cài đặt thư viện cần thiết
```python
!pip install transformers torch
```

## 3. Bài tập thực hành
### Bài 1: Khôi phục Masked Token (Masked Language Modeling)
Sử dụng pipeline `fill-mask` để dự đoán từ bị che trong câu: `Hanoi is the [MASK] of Vietnam.`

In [7]:
from transformers import pipeline
# Chỉ định framework PyTorch để tránh lỗi TensorFlow/Keras
mask_filler = pipeline("fill-mask", model="bert-base-uncased", framework="pt")
input_sentence = 'Hanoi is the [MASK] of Vietnam.'
predictions = mask_filler(input_sentence, top_k=5)
print(f'Câu gốc: {input_sentence}')
for pred in predictions:
    print(f"Dự đoán: '{pred['token_str']}' với độ tin cậy: {pred['score']:.4f}")
    print(f" -> Câu hoàn chỉnh: {pred['sequence']}")

c:\Users\ADMIN\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense

Câu gốc: Hanoi is the [MASK] of Vietnam.
Dự đoán: 'capital' với độ tin cậy: 0.9991
 -> Câu hoàn chỉnh: hanoi is the capital of vietnam.
Dự đoán: 'center' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the center of vietnam.
Dự đoán: 'birthplace' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the birthplace of vietnam.
Dự đoán: 'headquarters' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the headquarters of vietnam.
Dự đoán: 'city' với độ tin cậy: 0.0001
 -> Câu hoàn chỉnh: hanoi is the city of vietnam.


**Câu hỏi:**
1. Mô hình đã dự đoán đúng từ 'capital' không?
2. Tại sao các mô hình Encoder-only như BERT lại phù hợp cho tác vụ này?

**Trả lời:**
1. **Có**, mô hình đã dự đoán đúng từ 'capital' với độ tin cậy rất cao (99.91%). Kết quả top-5:
   - capital: 99.91%
   - center: 0.01%
   - birthplace: 0.01%
   - headquarters: 0.01%
   - city: 0.01%

2. BERT được huấn luyện với nhiệm vụ Masked Language Modeling, cho phép mô hình nhìn cả trái và phải của token bị che để dự đoán chính xác từ bị thiếu. Cơ chế **bidirectional** (hai chiều) giúp BERT hiểu ngữ cảnh đầy đủ: "Hanoi is the" (bên trái) và "of Vietnam" (bên phải), từ đó suy luận chính xác từ thiếu là "capital".

### Bài 2: Dự đoán từ tiếp theo (Next Token Prediction)
Sử dụng pipeline `text-generation` để sinh tiếp cho câu: `The best thing about learning NLP is`

In [8]:
from transformers import pipeline
# Chỉ định framework PyTorch để tránh lỗi TensorFlow
generator = pipeline("text-generation", model="gpt2", framework="pt")
prompt = "The best thing about learning NLP is"
generated_texts = generator(prompt, max_length=50, num_return_sequences=1)
print(f"Câu mồi: '{prompt}'")
for text in generated_texts:
    print("Văn bản được sinh ra:")
    print(text['generated_text'])

c:\Users\ADMIN\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ADMIN\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate 

Câu mồi: 'The best thing about learning NLP is'
Văn bản được sinh ra:
The best thing about learning NLP is that you always get to take what they have said and put it right there in a way that you feel like you understand."

He has been told by his team that "some of them didn't speak


**Câu hỏi:**
1. Kết quả sinh ra có hợp lý không?
2. Tại sao các mô hình Decoder-only như GPT lại phù hợp cho tác vụ này?

**Trả lời:**
1. **Có một phần hợp lý**. Văn bản được sinh ra:
   > "The best thing about learning NLP is that you always get to take what they have said and put it right there in a way that you feel like you understand."
   
   Câu này có cấu trúc ngữ pháp đúng và ý nghĩa liên quan đến việc hiểu ngôn ngữ. Tuy nhiên, nội dung hơi chung chung và phần cuối bị cắt đột ngột do giới hạn `max_length=50`. Đây là hạn chế của mô hình GPT-2 nhỏ và cần điều chỉnh tham số để có kết quả tốt hơn.

2. GPT được huấn luyện với nhiệm vụ **dự đoán từ tiếp theo** (next token prediction) dựa trên chuỗi đã có. Kiến trúc **unidirectional** (một chiều, từ trái sang phải) phù hợp cho sinh văn bản tự nhiên vì mô hình chỉ cần xem các từ trước đó để dự đoán từ kế tiếp, tương tự cách con người viết văn bản tuần tự.

### Bài 3: Tính toán vector biểu diễn của câu (Sentence Representation)
Tính vector biểu diễn cho câu `This is a sample sentence.` bằng phương pháp Mean Pooling với BERT.

In [9]:
import torch
from transformers import AutoTokenizer, AutoModel
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
sentences = ['This is a sample sentence.']
inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    outputs = model(**inputs)
last_hidden_state = outputs.last_hidden_state
attention_mask = inputs['attention_mask']
mask_expanded = attention_mask.unsqueeze(-1).expand(last_hidden_state.size()).float()
sum_embeddings = torch.sum(last_hidden_state * mask_expanded, 1)
sum_mask = torch.clamp(mask_expanded.sum(1), min=1e-9)
sentence_embedding = sum_embeddings / sum_mask
print('Vector biểu diễn của câu:')
print(sentence_embedding)
print('Kích thước của vector:', sentence_embedding.shape)

Vector biểu diễn của câu:
tensor([[-6.3874e-02, -4.2837e-01, -6.6779e-02, -3.8430e-01, -6.5784e-02,
         -2.1826e-01,  4.7636e-01,  4.8659e-01,  4.0658e-05, -7.4274e-02,
         -7.4741e-02, -4.7635e-01, -1.9773e-01,  2.4824e-01, -1.2162e-01,
          1.6678e-01,  2.1045e-01, -1.4576e-01,  1.2636e-01,  1.8635e-02,
          2.4640e-01,  5.7090e-01, -4.7014e-01,  1.3782e-01,  7.3650e-01,
         -3.3808e-01, -5.0330e-02, -1.6452e-01, -4.3517e-01, -1.2900e-01,
          1.6516e-01,  3.4004e-01, -1.4930e-01,  2.2422e-02, -1.0488e-01,
         -5.1916e-01,  3.2964e-01, -2.2162e-01, -3.4206e-01,  1.1993e-01,
         -7.0148e-01, -2.3126e-01,  1.1224e-01,  1.2550e-01, -2.5191e-01,
         -4.6374e-01, -2.7261e-02, -2.8415e-01, -9.9250e-02, -3.7017e-02,
         -8.9192e-01,  2.5005e-01,  1.5816e-01,  2.2701e-01, -2.8497e-01,
          4.5300e-01,  5.0940e-03, -7.9441e-01, -3.1008e-01, -1.7403e-01,
          4.3029e-01,  1.6816e-01,  1.0590e-01, -4.8987e-01,  3.1856e-01,
          3.

**Câu hỏi:**
1. Kích thước (chiều) của vector biểu diễn là bao nhiêu? Con số này tương ứng với tham số nào của mô hình BERT?
2. Tại sao chúng ta cần sử dụng `attention_mask` khi thực hiện Mean Pooling?

**Trả lời:**
1. **Kích thước: torch.Size([1, 768])**
   - **1**: batch size (1 câu)
   - **768**: chiều của hidden state
   
   Con số **768** tương ứng với tham số `hidden_size` của mô hình **bert-base-uncased**. Đây là số chiều của vector đầu ra từ mỗi lớp Transformer trong BERT. Các biến thể BERT khác có hidden_size khác nhau (ví dụ: bert-large có 1024).

2. `attention_mask` giúp loại bỏ ảnh hưởng của các **token padding** khi tính trung bình. Khi xử lý nhiều câu cùng lúc (batch), các câu ngắn sẽ được thêm padding để cùng độ dài. Nếu không dùng attention_mask, các token padding sẽ được tính vào trung bình, làm vector biểu diễn không chính xác. Attention_mask đảm bảo chỉ các token thực sự của câu được tính, cho kết quả đúng ngữ nghĩa.